# Zshot Example - Custom Components

### Install Zshot

In [1]:
!pip install git+https://github.com/IBM/zshot.git@main --quiet
#!pip install zshot --quiet

     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 953 kB 22.6 MB/s 
     |████████████████████████████████| 5.8 MB 51.0 MB/s 
     |████████████████████████████████| 452 kB 49.4 MB/s 
     |████████████████████████████████| 81 kB 8.0 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 182 kB 56.8 MB/s 
     |████████████████████████████████| 212 kB 70.7 MB/s 
     |████████████████████████████████| 132 kB 56.1 MB/s 
     |████████████████████████████████| 140 kB 71.3 MB/s 
     |████████████████████████████████| 7.6 MB 47.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


### Import Zshot 

In [2]:
import spacy
from zshot import PipelineConfig, displacy

### Create custom components

In Zshot it's easy to create new components like Mentions Extractor or Linkers. 

It's as simple as create a new class extending the base class (`MentionsExtractor` or `Linker`). You will have to implement the predict method, which will receive the SpaCy Documents and will return a list of `zshot.utils.data_models.Span` for each document.

In [3]:
from typing import Iterable
from spacy.tokens import Doc
from zshot import MentionsExtractor, Linker
from zshot.utils.data_models import Span


class CustomMentionExtractor(MentionsExtractor):
    def predict(self, docs: Iterable[Doc], batch_size=None):
        spans = [[Span(tok.idx, tok.idx + len(tok)) for tok in doc if "s" in tok.text] for doc in docs]
        return spans


class CustomLinker(Linker):
    def predict(self, docs: Iterable[Doc], batch_size=None):
        return [
            [Span(mention.start, mention.end, label='entity') for mention in doc._.mentions]
            for doc in docs
        ]

new_nlp = spacy.load("en_core_web_sm")
config = PipelineConfig(
    mentions_extractor=CustomMentionExtractor(),
    linker=CustomLinker()
)
new_nlp.add_pipe("zshot", config=config, last=True)

text = "CH2O2 is a chemical compound similar to Acetamide used in International \
Business Machines Corporation (IBM) to create new materials that act like PAGs."

doc = new_nlp(text)

displacy.render(doc, style="ent", jupyter=True)

/Users/marcosmartinez/Projects/AcceleratedDiscovery/ZeroShot/venvs/zshot/lib/python3.10/site-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
